## Import Libraries

In [15]:
import os    # Need this for environment variables
from crewai import Agent, Task, Crew,LLM    # Main CrewAI stuff for building agents
from langchain_community.tools import DuckDuckGoSearchRun   # For web searching
import time    # For adding delays
import random  # For random delays
from crewai.tools import tool # To create custom tools
from langchain_community.tools import DuckDuckGoSearchRun  # For web searching
from concurrent.futures import ThreadPoolExecutor # For running agents at same time
from IPython.display import display, Markdown # For showing results nicely




##  Disable CrewAI's telemetry to prevent data collection and errors

In [16]:
os.environ["CREWAI_TELEMETRY"] = "false"  # Turn off tracking
os.environ["CREWAI_TELEMETRY_DEBUG"] = "0"  # Turn off debug mode


## Load the  LLM


In [17]:
llm = LLM(
    model="groq/qwen/qwen3-32b",  # Using Qwen model from Groq
    api_key=os.getenv("GROQ_API_KEY"),  # Get my API key from environment
    api_base="https://api.groq.com/openai/v1",  # Groq API endpoint
    temperature=0.2,       # Keep responses focused, not too creative
    max_tokens=1000,       # Limit response length
    num_retries=5,         # Try 5 times if it fails
    request_timeout=90     # Wait up to 90 seconds
)

## Load  DuckduckGo Search tool

In [18]:
@tool("DuckDuckGo Search")       # Create a custom search tool for agents
def search_tool1(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    time.sleep(1.0 + random.random()*0.5) # Add random delay to avoid getting blocked
    return DuckDuckGoSearchRun().run(query)  # Do the actual search



## Topic for the LLM

In [ ]:
topic = "2024 Honda Civic review"    # What we want to research

SyntaxError: invalid syntax (2891766745.py, line 1)

## Defining all Agents

In [ ]:
# #  Agent 1: Reviewer A with an Edmunds bias
reviewer1 = Agent(
    role="Reviewer A (Edmunds bias)",  # This agent focuses on Edmunds
    goal="Search Edmunds for the topic and extract concise pros/cons.Only output the final answer without explaining the steps", # What it should do
    backstory="Automotive editor who summarizes objectively.",  # Give it personality
    tools=[search_tool1],  # Give it the search tool
    llm=llm,   # Use our LLM
    allow_delegation=False,  # Don't let it delegate to other agents
    verbose=False   # Keep output clean

)
#  Agent 2: Reviewer B with a Car and Driver bias
reviewer2 = Agent(
    role="Reviewer B (Car and Driver bias)",  # This one does Car and Driver
    goal="Search Car and Driver for the topic and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Performance-focused reviewer with balanced takes.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)
# Agent 3: Reviewer C with a Top Gear/other magazine bias
reviewer3 = Agent(
    role="Reviewer C (Top Gear / other mags bias)",  # This one does Top Gear etc
    goal="Search Top Gear or other reputable magazines and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Keeps things practical for buyers.",  # More practical focus
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)

#  Agent 4: Final Synthesizer, to combine the findings
synthesizer = Agent(
    role="Final Synthesizer",   # This guy combines everything
    goal="Merge the three reviewer notes into one clear, non-redundant verdict.Only output the final answer without explaining the steps",
    backstory="Senior editor who reconciles differing opinions.",   # The boss who makes final decisions
    tools=[],  # no web/tools; purely synthesis
    llm=llm,
    allow_delegation=False, 
    verbose=False
)


## Giving tasks descriptions 

In [ ]:
# Tasks for the three retrievers
t1 = Task(
    description=f"Topic: {topic}\nSearch site:edmunds.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer1,  # Give this task to the Edmunds reviewer
    expected_output="5 bullets + verdict + links" # What I want back
)
t2 = Task(
    description=f"Topic: {topic}\nSearch site:caranddriver.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer2,  # Give this to Car and Driver reviewer
    expected_output="5 bullets + verdict + links"
)
t3 = Task(
    description=f"Topic: {topic}\nSearch (site:topgear.com OR site:autocar.co.uk OR site:whatcar.com) {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer3,   #Give this to Top Gear reviewer
    expected_output="5 bullets + verdict + links"
)

# Function for running individual tasks in parallel

def run_task_in_parallel(task): 
    individual_crew = Crew(      
        agents=[task.agent],      # Put the agent in its own crew
        tasks=[task],             # Give it the task to do
        verbose=False             # Don't show too much detail
    )
    return individual_crew.kickoff()       # Start the work and return result

# Running t1, t2, t3 in parallel using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=3) as executor:        # Start 3 workers at the same time
    future_t1 = executor.submit(run_task_in_parallel, t1)  # Start Edmunds search
    future_t2 = executor.submit(run_task_in_parallel, t2)  # Start Car & Driver search
    future_t3 = executor.submit(run_task_in_parallel, t3)  # Start Top Gear search
    
    # Get results from parallel execution
    result_t1 = future_t1.result()   # Wait for Edmunds result
    result_t2 = future_t2.result()   # Wait for Car & Driver result
    result_t3 = future_t3.result()   # Wait for Top Gear result

#  t_final task description
t_final = Task(
    description=(
        f"Based on these three reviews:\n\n"
        f"Review 1 (Edmunds): {result_t1}\n\n"
        f"Review 2 (Car & Driver): {result_t2}\n\n" 
        f"Review 3 (Top Gear): {result_t3}\n\n"
        "Combine the three reviewer outputs into:\n"
        "- 4-6 consensus bullets (no duplicates)\n"
        "- One verdict paragraph (3-5 sentences) for buyers\n"
        "- 'Where reviewers agree' and 'Where they differ' (1-2 bullets each)\n"
        "Keep it neutral, specific, and text-only."
    ),
    agent=synthesizer,      # Give this to the synthesizer agent
    expected_output="Bullets + verdict paragraph + agreement/differences"   # What I expect back
)

# Final synthesis
final_crew = Crew(
    agents=[synthesizer],  # Just the synthesizer agent
    tasks=[t_final],       # Give it the final task
    verbose=False           # Keep it quiet
)

final_result = final_crew.kickoff()    # Run the synthesis and get final result

# Displaying results
results = [result_t1, result_t2, result_t3, final_result]   # Put all results in a list

# Names for each agent
agent_roles = ["Reviewer A (Edmunds bias)", "Reviewer B (Car and Driver bias)", "Reviewer C (Top Gear / other mags bias)", "Final Synthesizer"]

# Loop through and show each result with its agent name
for i, (result, role) in enumerate(zip(results, agent_roles)):
    display(Markdown(f"### {i}: {role}\n\n{result}")) # Display each result nicely formatted

### 0: Reviewer A (Edmunds bias)

- **Pros**: 2024 Honda Civic offers a refined 1.5L turbo engine with improved fuel efficiency (32 mpg city/42 mpg highway), updated 12.3-inch touchscreen with Apple CarPlay/Android Auto, available all-wheel drive, enhanced safety tech (Honda Sensing 3.0), and a more premium interior with soft-touch materials.
- **Cons**: Base models lack wireless charging, suspension feels firm on rough roads, limited rear legroom for taller passengers, infotainment response time lags occasionally, and cargo space remains modest at 15.1 cubic feet.
- **Verdict**: The 2024 Civic remains a strong contender in the compact sedan segment, balancing practicality, reliability, and tech upgrades, though it falls short for those needing more space or a plush ride.
- **Links**: 
  https://www.edmunds.com/2024-honda-civic-review-rating/
  https://www.edmunds.com/honda/civic/2024/

### 1: Reviewer B (Car and Driver bias)

- **Pros**: 2024 Honda Civic offers a refined 1.5L turbocharged engine with 182 hp, improved fuel efficiency (32 mpg city/42 mpg highway), and a sleek, tech-focused interior with a 10.2-inch touchscreen.  
- **Cons**: Base models lack standard features like Apple CarPlay and automatic emergency braking, while the suspension feels firm on rough roads.  
- **Verdict**: A well-rounded compact car with strong performance and efficiency, but entry-level trims feel under-equipped compared to competitors.  

Links:  
https://www.caranddriver.com/honda/civic/reviews/2024-honda-civic-first-drive-review  
https://www.caranddriver.com/news/2024-honda-civic-sedan-and-hatchback-get-minor-updates

### 2: Reviewer C (Top Gear / other mags bias)

**2024 Honda Civic Review**  
- **Pros/Cons**:  
  - 1.5T turbo engine offers sharp responsiveness and fuel efficiency.  
  - Updated interior with better tech (Honda’s new infotainment system) but some creaky plastics.  
  - Agile handling and precise steering, though ride quality can feel firm.  
  - Hybrid variant delivers excellent real-world mpg without sacrificing performance.  
  - Competitive pricing, but optional tech packages add significant cost.  
**Verdict**: A well-rounded compact car with strong dynamics and efficiency, though interior build quality lags slightly behind rivals.  
**Links**:  
https://www.topgear.com/car-reviews/honda/civic-2024-review  
https://www.autocar.co.uk/car-reviews/honda/civic-2024-review

### 3: Final Synthesizer

<think>
Okay, let's tackle this. I need to combine the three reviews into a concise summary with specific bullets, a verdict, and sections on where they agree and differ. First, I'll go through each review's pros and cons to identify common points.

Review 1 (Edmunds) mentions the 1.5L turbo engine with improved fuel efficiency (32/42 mpg), a 12.3-inch touchscreen with Apple CarPlay/Android Auto, all-wheel drive availability, Honda Sensing 3.0 safety tech, and a premium interior. Cons include no wireless charging on base models, firm suspension, limited rear legroom, laggy infotainment, and modest cargo space.

Review 2 (Car & Driver) highlights the same engine and fuel efficiency, a 10.2-inch touchscreen. Cons are missing Apple CarPlay and automatic emergency braking on base models, and firm suspension. They note entry-level trims are under-equipped.

Review 3 (Top Gear) talks about the 1.5T engine's responsiveness and efficiency, updated interior with new infotainment but creaky plastics, agile handling, firm ride, hybrid variant's mpg, and optional tech packages adding cost. Verdict mentions dynamics and efficiency but interior build quality lags.

Now, for consensus bullets. The engine and fuel efficiency are mentioned in all three. All agree on the firm suspension. The interior tech is a pro in all but with some cons like lag or creaky plastics. Handling/agility is a pro in Top Gear and Edmunds mentions all-wheel drive. Cargo space and rear legroom are cons in Edmunds and Car & Driver. Optional features adding cost is in Top Gear and Edmunds mentions base models lacking some features.

Verdict needs to balance the positives like performance, efficiency, tech, and handling, while noting the cons like base model features, interior build, and space. 

Agreement points: All agree on the engine's efficiency and firm ride. They also agree on the interior tech having issues. Differences: Edmunds and Car & Driver mention base models lacking features, while Top Gear talks about optional packages. Edmunds and Car & Driver note space issues, Top Gear mentions hybrid variant. 

Need to make sure bullets are unique and not duplicated. Let me check each point again. Engine and fuel efficiency are common. Suspension firmness is in all. Interior tech pros and cons. Handling is in two reviews. Cargo space and rear legroom in two. Optional features in one. 

Now, structure the bullets into 4-6 points. Maybe:

1. 1.5L turbo engine with 32/42 mpg.
2. Updated infotainment (sizes vary) with some lag or creaky plastics.
3. Agile handling and precise steering, but firm ride.
4. Base models lack wireless charging, Apple CarPlay, and safety features; optional packages add cost.
5. Limited rear legroom and modest cargo space (15.1 cu ft).

Verdict: The 2024 Civic is a strong compact with good efficiency and handling, but base models are under-equipped and interior build quality is mixed. 

Agreement: Engine efficiency, firm suspension, interior tech issues. Differences: Base features vs optional packages, space vs hybrid variant.

Need to make sure the verdict is concise and covers the main points. Also check for any missed details. Let me put it all together now.
</think>

**Consensus Bullets:**  
- **1.5L turbo engine delivers 182 hp and 32/42 mpg fuel efficiency**, with sharp responsiveness and refined performance.  
- **Updated infotainment system** (10.2–12.3-inch touchscreen) includes Apple CarPlay/Android Auto but suffers from occasional lag or creaky plastics.  
- **Agile handling and precise steering** with a firm ride that may discomfort on rough roads.  
- **Base models lack standard features** like wireless charging, Apple CarPlay, and automatic emergency braking, while optional tech packages add cost.  
- **Limited rear legroom and modest cargo space** (15.1 cu ft) may disappoint those needing more room.  

**Verdict:**  
The 2024 Honda Civic excels as a practical, efficient compact sedan with a potent turbo engine, agile dynamics, and tech-focused interior. Its fuel efficiency and driving engagement make it a top choice for daily commuters, but base trims feel under-equipped compared to competitors, and interior build quality lags slightly. While the hybrid variant offers exceptional mpg, buyers prioritizing space or a plush ride may find it lacking.  

**Where Reviewers Agree:**  
- All agree on the engine’s strong performance and fuel efficiency.  
- Consensus on the firm suspension and interior tech’s mixed quality (pros for features, cons for lag